In [1]:
"""
v1.3

4/16/21

inspiration: https://www.youtube.com/watch?v=uWCqlo893tk

add stocks to the list that arent included like LOW

inflation target = 2%+1

add dividend reinvestment

add previous 10 y yield actual (1k of price 10y ago)

add a piece that compares 1k invested today and its projected yield in 10 y

add book value, fair value, price, market cap


step 1: identify companies that are good buys (partial check (add valuations like book value and fair value maybe 
    from morningstar))Requirements: beat long term inflation, good company, good value. Compare all buying 
    opportunities i.e. expected return on stocks, crypto, houses, etc

step 2: figure out when are likely good times to buy them, potential is using bollinger bands to see when certain 
    stocks are at their low

step 3: automate the buying and selling of options to hit target prices. sell call options on stocks i own for 
    linear valuations and after high tax events (like potentially keep for a year)(work with tax advisor)



Requirements: beat long term inflation, good company, good value
"""

'\nv1.3\n\n4/16/21\n\ninspiration: https://www.youtube.com/watch?v=uWCqlo893tk\n\nadd stocks to the list that arent included like LOW\n\ninflation target = 2%+1\n\nadd dividend reinvestment\n\nadd previous 10 y yield actual (1k of price 10y ago)\n\nadd a piece that compares 1k invested today and its projected yield in 10 y\n\nadd book value, fair value, price, market cap\n\n\nstep 1: identify companies that are good buys (partial check (add valuations like book value and fair value maybe \n    from morningstar))Requirements: beat long term inflation, good company, good value. Compare all buying \n    opportunities i.e. expected return on stocks, crypto, houses, etc\n\nstep 2: figure out when are likely good times to buy them, potential is using bollinger bands to see when certain \n    stocks are at their low\n\nstep 3: automate the buying and selling of options to hit target prices. sell call options on stocks i own for \n    linear valuations and after high tax events (like potential

In [2]:
import pandas as pd
import numpy as np
import import_ipynb
import dividend_detective_scrape

import yfinance as yf

import datetime as dt

from datetime import datetime

from numpy import mean

importing Jupyter notebook from dividend_detective_scrape.ipynb


In [3]:
def trim_div_data(stockObj):
    actns = stockObj.actions
    for i,r in actns.iterrows():
        if r['Dividends'] > 0:
            first_div_date = i
            break

    actns = actns.loc[first_div_date:]
    return actns, first_div_date

In [4]:
def add_div_data_details(actns, first_div_date):
    actns['days_since_first_div'] = (actns.index-first_div_date).days
    actns['year'] = actns.index.year
    
    h = []
    l = []
    for i,r in actns.iterrows():
        prices = stockObj.history(start=i,end = i+pd.DateOffset(1)).iloc[0]
        h.append(prices['High'])
        l.append(prices['Low'])

    actns['Stock_High'] = h
    actns['Stock_Low'] = l
    
    actns['day_div_yield'] = actns['Dividends']/actns['Stock_High']
    return actns

In [5]:
def get_ind_div_data(stockObj):
    df, first_div_date = trim_div_data(stockObj)
    return add_div_data_details(df, first_div_date)

In [6]:
"""

then learn how long it will take to beat other investments...

"""

'\n\nthen learn how long it will take to beat other investments...\n\n'

In [7]:
def get_historical_div_info(stockObj):
    df = stockObj.actions
    df['year'] = df.index.year
    df = df[df['Dividends']>0]
    df = df.drop(columns=['Stock Splits'])
    return df

In [8]:
def group_divs_by_year(df):
#     combine same year divs and rename column
    ann_df = df.groupby(['year']).agg(
        {'Dividends': np.sum}).rename(columns=
        {'Dividends':'dividends'})
    start_year = ann_df.index[0]
    end_year = ann_df.index[len(ann_df)-1]+1
    ann_df = ann_df.reindex(list(range(start_year,end_year)), fill_value=0)
    return ann_df

In [9]:
def add_consistancy_coeff(df):
    consistancy_coeff = [1/(2**(x+1)) for x in range(len(df))][::-1]
    year_progress = datetime.now().timetuple().tm_yday/365
    consistancy_coeff[-1] = consistancy_coeff[-1]*year_progress
    
    df['div_consistency_score'] = consistancy_coeff
    return df

In [10]:
def calculate_div_consistency_score(df):
    
    df = add_consistancy_coeff(df)

    # default highest div is the first div
    highest_div = df.iloc[0]['dividends']
    
#     add another weight to the last cell where i get the percentage of the year we've been in and multiply it by that
# exclude 

    for i,r in df[1:-1].iterrows():      
#         i need to think about a better way to do this because right now
#         it compares the wrong two pieces
        if r['dividends'] < df.loc[i-1]['dividends']:
            mult = r['div_consistency_score']
            div_perc = r['dividends']/highest_div
            df.at[i, 'div_consistency_score'] = mult * div_perc
        else:
            highest_div = r['dividends']
    
    return df['div_consistency_score'].sum()

In [11]:
def structure_df(df, years):
    df = df[:-years-3:-1]
    df = df[1:]
    return df

In [12]:
def get_div_deltas(df):
    
    div_deltas = []
        
    for i,r in df[:-1].iterrows():
        c = float(df.loc[i]['dividends'])
        p = float(df.loc[i-1]['dividends'])

        div_deltas.append((c-p)/p)
    
    return div_deltas

In [13]:
def get_div_growth_rate(df, years):
    if years >= len(df)-1:
        return 0
    else:
        df = structure_df(df, years)
        
        div_deltas = get_div_deltas(df)
        
    return mean(div_deltas)
            

In [14]:
def get_div_analysis(stockObj):

    df = get_historical_div_info(stockObj)
    ann_df = group_divs_by_year(df)
    
    div_consistacy_score = calculate_div_consistency_score(ann_df)
    
    div_growth_5y = get_div_growth_rate(ann_df,5)
    div_growth_10y = get_div_growth_rate(ann_df,10)
    div_growth_15y = get_div_growth_rate(ann_df,15)

    return div_consistacy_score, div_growth_5y, div_growth_10y, div_growth_15y

In [15]:
def get_yield(div_yield, div_growth, years=10, investment=1000):
    roi = -investment
    dy = div_yield
    dg = div_growth
    
    for i in range(years):
        roi = roi + investment*dy
        dy = dy*(1+dg)
    
    return roi

In [16]:
def get_all_div_analysis(stockList):
    for index, row in stockList.iterrows():
        try:
            stockObj = yf.Ticker(row['ticker'])
            
            div_analysis = get_div_analysis(stockObj)
            
            div_consis, div_growth_5y, div_growth_10y, div_growth_15y = get_div_analysis(stockObj)
            
        except:
            div_consis = 0.0
            div_growth_5y = 0.0
            div_growth_10y = 0.0
            div_growth_15y = 0.0
            
#         1k 10y div yield projections go here

        stockList.at[index, 'yield_1k_in_10y'] = round(get_yield(row['div_yield'], div_growth_5y),2)
            
        stockList.at[index, 'div_consistency_score'] = div_consis
        stockList.at[index, 'div_growth_5y'] = div_growth_5y
        stockList.at[index, 'div_growth_10y'] = div_growth_10y
        stockList.at[index, 'div_growth_15y'] = div_growth_15y
        
    return stockList

In [17]:
def add_div_analysis(df):
    df['yield_1k_in_10y'] = 0.0
    df['div_consistency_score'] = 0.0
    df['div_growth_5y'] = 0.0
    df['div_growth_10y'] = 0.0
    df['div_growth_15y'] = 0.0
    df = get_all_div_analysis(df)
    return df

In [18]:
def get_div_stocks(min_div = .03, min_consistency=.5):
    df = dividend_detective_scrape.get_data()
    
    df['div_yield'] = df['div_yield'] * 0.01
    
    df = df[df['div_yield']>min_div]
    
    df = add_div_analysis(df)   
    
    df = df[df['div_consistency_score']>min_consistency]
    df = df.sort_values('div_consistency_score', ascending=False)
    
    return df
    

In [19]:
def get_select_div_stocks(df, div_5y_growth = 0.03, growing_div = True, div_5y_growth_max = 1, min_req_10y_yield = 0):
    
    s = 'div_growth_5y >= '+ str(div_5y_growth)
    df = df.query(s)
    
    if growing_div:
        df = df.query('div_growth_5y > div_growth_10y & div_growth_10y > div_growth_15y')
        
    df = df[df['div_growth_5y']<div_5y_growth_max]
    df = df[df['yield_1k_in_10y']>min_req_10y_yield]
    df = df.sort_values('yield_1k_in_10y', ascending=False)
    
    return df
    

In [20]:
df = get_div_stocks(min_div = .001)

- CWEN.A: No data found, symbol may be delisted
- CRD.A: No data found, symbol may be delisted
- EV: No data found, symbol may be delisted
- QEP: No data found, symbol may be delisted
- TCP: No data found, symbol may be delisted
- WYND: No data found, symbol may be delisted


In [21]:
df

,ticker,name,annual_div,div_yield,yield_1k_in_10y,div_consistency_score,div_growth_5y,div_growth_10y,div_growth_15y
172,KO,Coca-Cola,1.64,0.032,-607.71,0.691781,0.044489,0.064502,0.074580
178,CL,Colgate-Palmolive,1.76,0.022,-746.24,0.691781,0.031335,0.056396,0.080478
153,CVX,Chevron,5.16,0.056,-332.07,0.691781,0.038580,0.062408,0.075703
265,XOM,Exxon Mobil,3.48,0.069,-176.46,0.691781,0.038724,0.072788,0.078103
61,ADP,Automatic Data Processing,3.72,0.022,-595.87,0.691781,0.129439,0.117566,0.142193
...,...,...,...,...,...,...,...,...,...
315,GILT,Gilat Satellite Networks,0.99,0.047,-530.00,0.516781,0.000000,0.000000,0.000000
342,THG,Hanover Insurance Gr,2.80,0.024,3076.96,0.510079,0.585667,0.348185,0.342679
331,GPMT,Granite Point Mortgage,0.80,0.077,-230.00,0.507257,0.000000,0.000000,0.000000
128,CADE,Cadence Bancorp,0.60,0.032,-680.00,0.504281,0.000000,0.000000,0.000000


In [22]:
fdf = get_select_div_stocks(df, div_5y_growth=0)

In [23]:
fdf

,ticker,name,annual_div,div_yield,yield_1k_in_10y,div_consistency_score,div_growth_5y,div_growth_10y,div_growth_15y
253,EQR,Equity Residential,2.41,0.036,18457.69,0.665295,0.845343,0.465256,0.300288
299,BEN,Franklin Resources,1.12,0.041,9349.12,0.595508,0.671836,0.640524,0.608405
327,GFI,Gold Fields,0.19,0.020,6087.35,0.625708,0.747778,0.346293,0.278804
342,THG,Hanover Insurance Gr,2.80,0.024,3076.96,0.510079,0.585667,0.348185,0.342679
244,EBF,Ennis,0.90,0.046,1384.90,0.663316,0.339560,0.269890,0.179926
181,CMA,Comerica,2.72,0.044,796.08,0.691662,0.291306,0.288029,0.151779
127,COG,Cabot Oil & Gas,0.40,0.022,758.36,0.691781,0.427689,0.330511,0.257463
237,EIG,Employers Holdings,1.00,0.032,723.51,0.691750,0.347273,0.173636,0.000000
239,ENB,Enbridge,2.61,0.076,591.31,0.666693,0.156295,0.142153,0.135009
511,TSN,Tyson Foods,1.78,0.027,451.08,0.663461,0.346836,0.308407,0.205605


In [24]:
df.shape

(503, 9)

In [25]:
df[df['ticker']=='XOM']

,ticker,name,annual_div,div_yield,yield_1k_in_10y,div_consistency_score,div_growth_5y,div_growth_10y,div_growth_15y
265,XOM,Exxon Mobil,3.48,0.069,-176.46,0.691781,0.038724,0.072788,0.078103


In [26]:
fdf[fdf['div_consistency_score']>.65]

,ticker,name,annual_div,div_yield,yield_1k_in_10y,div_consistency_score,div_growth_5y,div_growth_10y,div_growth_15y
253,EQR,Equity Residential,2.41,0.036,18457.69,0.665295,0.845343,0.465256,0.300288
244,EBF,Ennis,0.90,0.046,1384.90,0.663316,0.339560,0.269890,0.179926
181,CMA,Comerica,2.72,0.044,796.08,0.691662,0.291306,0.288029,0.151779
127,COG,Cabot Oil & Gas,0.40,0.022,758.36,0.691781,0.427689,0.330511,0.257463
237,EIG,Employers Holdings,1.00,0.032,723.51,0.691750,0.347273,0.173636,0.000000
239,ENB,Enbridge,2.61,0.076,591.31,0.666693,0.156295,0.142153,0.135009
511,TSN,Tyson Foods,1.78,0.027,451.08,0.663461,0.346836,0.308407,0.205605
268,FMNB,Farmers National Banc,0.44,0.032,359.14,0.690924,0.299306,0.166320,0.049213
482,SNV,Synovus Finl,1.32,0.032,186.32,0.691669,0.271905,0.182151,0.086684
526,UVV,Universal,3.08,0.060,164.03,0.678022,0.140670,0.085656,0.068107


In [27]:
option_terms = {
    'buy call': 'allows you to buy stock at strike price (above current price) . allows market to sell you at strike price. hedge strategy: pay a little now '
    'sell call': 'allows you to sell stock at strike price (above current price). allows market to buy from you at strike price'
    'buy put': 'allows you to sell stock at strike price (below current price)'
    'sell put': 'allows you buy stock at strike price (below current price)'
}

SyntaxError: invalid syntax (<ipython-input-27-aef18d81f95a>, line 3)